In [4]:
!pip install requests

In [7]:
import json
import requests

def mrc_run(input_data) :
    response = requests.post('http://192.168.0.132:8888/mrc', json = input_data)
    response_data = json.loads(response.text)
    
    return response_data

context = "나는 커피를 좋아합니다"

question = "나는 무엇을 좋아합니까?"


input_data = { 'context' : [context], 'question' : [question]}

mrc_run(input_data)



[{'score': 15.521124362945557, 'position': 2, 'answer': '커피'}]

In [51]:
import os
import pandas as pd
import json
from collections import OrderedDict
import datetime

class Combine_sentence :
    
    def input_xlsx_path(self) :
        print('='*70)
        self.xlsx_path = 'D:\\DEMO\\xlsx'
        while True :
            self.xlsx_path = input('나라 데이터가 존재하는 xlsx 경로를 입력해주세요.\n예시 경로 : {} \n경로 입력 : '.format(self.xlsx_path))
            self.xlsx_path = self.xlsx_path.strip()
            
            if self.xlsx_path == '0' :
                break
                
            if os.path.isdir(self.xlsx_path) :
                self.country_menu()
                break
            else :
                print('0을 입력하면 종료할 수 있습니다.\n')
            
    def country_menu(self) :
        print('='*70)
        path_list = os.listdir(self.xlsx_path)
        self.text = ''
        self.country_list = []
        country_menu = ''
        for i in path_list :
            if '.xlsx' in i or '.xls' in i :
                self.country = i.split('.')[0]
                self.country_list.append(self.country)

        for i in range(len(self.country_list)) :
            if(i%4 == 0) :
                self.text = self.text+'\n'
            self.text = self.text+str(i)+'. '+self.country_list[i]+'  \t\t'
            
        print(self.text)
        
    def input_country(self) :
        print('='*70)
        while True :
            num = input('나라번호 입력 : ')
            
            if num == '0' :
                break
            
            if num.isdecimal() :
                num = int(num)
                if num >=0 and num < len(self.country_list) :
                    print('*** {}번 {} 나라 선택 ***'.format(num, self.country_list[num]))
                    break
                else :
                    print('0을 입력하면 종료할 수 있습니다.\n')
            else :
                print('0을 입력하면 종료할 수 있습니다.\n')
     
        self.country = self.country_list[num]
        return self.country
    
    
    def open_xlsx(self, country, xlsx_path) :
        print('='*70)
        df = pd.read_excel(xlsx_path+'\\'+country+'.xlsx', sheet_name = '질문지')
        sent_count = len(df.index)
        
        self.questions = []
        self.mystr = ''
        
        try :
            w = df['sentence']
        except KeyError :
            print('xlsx 파일의 내용이 현 프로그램과 관련이 없습니다.')
        else :
            for i in range(len(w)) :
                if '?' in str(w[i]) :
                    self.questions.append(str(w[i]))
                else :
                    self.mystr += str(w[i]) + '\n'
    
    def write_json(self, file_data) :
        json_path = os.getcwd()+'\\xlsx\\'
        if not os.path.isdir(json_path) :
            print('{}경로가 존재하지 않아서 폴더를 생성합니다.\n'.format(json_path))
            os.makedirs(json_path)
            
        path =  json_path+self.country + '.json'
        if os.path.isfile(path) :
            date = datetime.datetime.now()
            now = datetime.datetime.strftime(date, '%Y년%m월%d일_%H시%M분')
            self.name = self.country+'_'+now
            print('동일한 파일이 존재하여 {} 이름으로 저장합니다.\n'.format(self.name))
            path =  json_path+self.name + '.json'
            
        with open(path, 'w', encoding='utf-8') as make_file :
            json.dump(file_data, make_file, ensure_ascii=False, indent='\t')
            
        print('{} 에 json 파일이 저장했습니다.\n'.format(path))
    
    def json_set(self) :
        file_data = OrderedDict()
        file_data['context'] = self.mystr
        file_data['question'] = self.questions
        
        return file_data
        
    
    def mrc_run(self, input_data) :
        print('='*70)
        for i in range(len(self.questions)) :
            print(self.questions[i])
            input_data = { 'context' : [self.mystr], 'question' : [self.questions[i]]}
            response = requests.post('http://192.168.0.132:8888/mrc', json = input_data)
            response_data = json.loads(response.text)
            print(response_data)
            

    def sys_run(self) :
        self.input_xlsx_path()
        country = self.input_country()
        self.open_xlsx(country, self.xlsx_path)
        
        file_data = self.json_set()
        self.write_json(file_data)
        self.mrc_run(file_data)
        print('='*70)
        
instance = Combine_sentence()
instance.sys_run()
        

나라 데이터가 존재하는 xlsx 경로를 입력해주세요.
예시 경로 : D:\DEMO\xlsx 
경로 입력 : D:\DEMO\xlsx 

0. 대한민국  		1. 인도  		2. 이란  		3. 베트남  		
4. 한국  		5. 태국  		6. 이스라엘  		7. 말레이시아  		
8. 필리핀  		9. 대만  		10. 싱가포르  		11. 인도네시아  		
12. 터키  		13. 북한  		14. 중국  		15. 러시아  		
나라번호 입력 : 2
*** 2번 이란 나라 선택 ***
동일한 파일이 존재하여 이란_2022년04월13일_16시30분 이름으로 저장합니다.

C:\Users\pc201\Project_Study\xlsx\이란_2022년04월13일_16시30분.json 에 json 파일이 저장했습니다.

이란은 어디에 있는가?
[{'score': 14.383740425109863, 'position': 6, 'answer': '서남아시아'}]
이란은 무슨 공화국인가?
[{'score': 16.10991907119751, 'position': 16, 'answer': '이슬람 공화국'}]
1935년전에 페르시아로 알려진 공화국은?
[{'score': 18.852876663208008, 'position': 34, 'answer': '이란 이슬람 공화국'}]
공식 명칭은 무엇인가?
[{'score': 21.261019706726074, 'position': 34, 'answer': '이란 이슬람 공화국'}]
1935년 전 이란 명칭은?
[{'score': 17.541112899780273, 'position': 58, 'answer': '페르시아'}]
군주제는 약 몇 년만에 폐지되었는가?
[{'score': 17.46131706237793, 'position': 2338, 'answer': '1736년'}]
이란은 현재 어떤 국가인가?
[{'score': 20.42590618133545, 'position': 130, 'answer': '공화제 국가'}

KeyboardInterrupt: 

In [23]:
fffff = '2'

if fffff.isdecimal() :
    print('아?')

아?


In [56]:
import os
import pandas as pd
import json
from collections import OrderedDict
import datetime

class Combine_sentence :
    
    def input_xlsx_path(self) :
        print('='*70)
        self.xlsx_path = 'D:\\DEMO\\xlsx'
        while True :
            self.xlsx_path = input('나라 데이터가 존재하는 xlsx 경로를 입력해주세요.\n예시 경로 : {} \n경로 입력 : '.format(self.xlsx_path))
            self.xlsx_path = self.xlsx_path.strip()
            
            if self.xlsx_path == '0' :
                break
                
            if os.path.isdir(self.xlsx_path) :
                self.country_menu()
                break
            else :
                print('0을 입력하면 종료할 수 있습니다.\n')
            
    def country_menu(self) :
        print('='*70)
        path_list = os.listdir(self.xlsx_path)
        self.text = ''
        self.country_list = []
        country_menu = ''
        for i in path_list :
            if '.xlsx' in i or '.xls' in i :
                self.country = i.split('.')[0]
                self.country_list.append(self.country)

        for i in range(len(self.country_list)) :
            if(i%4 == 0) :
                self.text = self.text+'\n'
            self.text = self.text+str(i)+'. '+self.country_list[i]+'  \t\t'
            
        print(self.text)
        
    def input_country(self) :
        print('='*70)
        while True :
            num = input('나라번호 입력 : ')
            
            if num == '0' :
                break
            
            if num.isdecimal() :
                num = int(num)
                if num >=0 and num < len(self.country_list) :
                    print('*** {}번 {} 나라 선택 ***'.format(num, self.country_list[num]))
                    break
                else :
                    print('0을 입력하면 종료할 수 있습니다.\n')
            else :
                print('0을 입력하면 종료할 수 있습니다.\n')
     
        self.country = self.country_list[num]
        return self.country
    
    
    def open_xlsx(self, country, xlsx_path) :
        print('='*70)
        self.questions = []
        self.mystr = ''
        
        try :
            df = pd.read_excel(xlsx_path+'\\'+country+'.xlsx', sheet_name = '질문지')
            sent_count = len(df.index)
            w = df['sentence']
        except KeyError :
            print('xlsx 파일의 내용이 현 프로그램과 관련이 없습니다.')
            return False
        except ValueError:
            print('xlsx 파일의 내용이 현 프로그램과 관련이 없습니다.')
            return False
        else :
            for i in range(len(w)) :
                if '?' in str(w[i]) :
                    self.questions.append(str(w[i]))
                else :
                    self.mystr += str(w[i]) + '\n'
    
        return True
    
    def write_json(self, file_data) :
        json_path = os.getcwd()+'\\xlsx\\'
        if not os.path.isdir(json_path) :
            print('{}경로가 존재하지 않아서 폴더를 생성합니다.\n'.format(json_path))
            os.makedirs(json_path)
            
        path =  json_path+self.country + '.json'
        if os.path.isfile(path) :
            date = datetime.datetime.now()
            now = datetime.datetime.strftime(date, '%Y년%m월%d일_%H시%M분')
            self.name = self.country+'_'+now
            print('동일한 파일이 존재하여 {} 이름으로 저장합니다.\n'.format(self.name))
            path =  json_path+self.name + '.json'
            
        with open(path, 'w', encoding='utf-8') as make_file :
            json.dump(file_data, make_file, ensure_ascii=False, indent='\t')
            
        print('{} 에 json 파일이 저장했습니다.\n'.format(path))
    
    def json_set(self) :
        file_data = OrderedDict()
        file_data['context'] = self.mystr
        file_data['question'] = self.questions
        
        return file_data
        
    
    def mrc_run(self, input_data) :
        print('='*70)
        for i in range(len(self.questions)) :
            print(self.questions[i])
            input_data = { 'context' : [self.mystr], 'question' : [self.questions[i]]}
            response = requests.post('http://192.168.0.132:8888/mrc', json = input_data)
            response_data = json.loads(response.text)
            print(response_data)
            

    def sys_run(self) :
        self.input_xlsx_path()
        country = self.input_country()
        if self.open_xlsx(country, self.xlsx_path) :
        
            file_data = self.json_set()
            self.write_json(file_data)
            self.mrc_run(file_data)
            print('='*70)
        
instance = Combine_sentence()
instance.sys_run()
        

나라 데이터가 존재하는 xlsx 경로를 입력해주세요.
예시 경로 : D:\DEMO\xlsx 
경로 입력 : C:\Users\pc201\Untitled Folder\xlsx

0. 엑셀 공부하자  		1. 졸리다  		
나라번호 입력 : 1
*** 1번 졸리다 나라 선택 ***
xlsx 파일의 내용이 현 프로그램과 관련이 없습니다.
